In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
import cv2
import regex as re
import math
import time
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib import patches

from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout, concatenate, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

In [ ]:
data_path = '/kaggle/input/kuzushiji-recognition'
unicode_translation = os.path.join('/kaggle/input/kuzushiji-recognition','unicode_translation.csv')
train_csv_path = os.path.join('/kaggle/input/kuzushiji-recognition','train.csv')
train_image_path = os.path.join('/kaggle/input/kuzushiji-recognition','train_images')

In [ ]:
train_df = pd.read_csv(train_csv_path)
print(train_df)

In [ ]:
unicodes = pd.read_csv(unicode_translation)
print(unicodes['Unicode'])

In [ ]:
def split_label_set(label_set):
    label_item = split_label(label_set)
    label_items = np.array(label_item).reshape(-1, 5)
    return label_items

def split_label(label):
    label = str(label)
    return label.split(' ')

def image_path(id):
    id = str(id)
    if '.jpg' not in id:
        return train_image_path + '/' + id + '.jpg'
    
def show_image(image_id, label=None):
    img = cv2.imread(os.path.join(image_path(image_id)))
    if label:
        unicode, x, y, w, h = splitLabel(label)
        print("w: {}, h:{}".format(w,h))
        img = img[y:y+h, x:x+w]
        plt.title(unicode)
    plt.imshow(img)

def show_image_with_labels(df):
    img = cv2.imread(os.path.join(image_path(df['image_id'])))
    labels = split_label_set(df['labels'])
    plt.imshow(img)
    for label in labels:
        unicode, x, y, w, h = split_label(label)
        plt.gca().add_patch(patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none'))

In [ ]:
width = []
for i in range(500):
    df = train_df.loc[i]
    img = cv2.imread(os.path.join(image_path(df['image_id'])))
    width = 
    print(np.shape(img))

In [ ]:
def get_mask(img, x, y, width, height):
    #load the cropped area and apply an Otsu threshold
    cropped_img = np.array(img[y:y+height,x:x+width,:])
    blurred_img = cv2.GaussianBlur(cropped_img,(5,5),0)
    img_gray = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2GRAY)
    ret, otsu = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    #Place back the cropped area into a mask with the original image size
    img_height, img_width = img.shape[:2]
    img_mask = np.full((img_height,img_width),0)
    img_mask[y:y+height,x:x+width] = otsu

    return img_mask

def get_mask_for_df(df):
    img = cv2.imread(os.path.join(image_path(df['image_id'])))
    mask = np.zeros((img.shape[0], img.shape[1], 2), dtype='float32')
    labels = df['labels']
    if isinstance(labels, str):
        labels = split_label_set(labels)

        mask = np.full((img.shape[:2]),0)
        for i, label in enumerate(labels):
            x, y, w, h = (int(t) for t in label[1:])
            mask = mask + get_mask(img, x, y, w, h)
    return mask

# for i in range(len(train_df.index)):
for i in range(1):
    df = train_df.iloc[i]
    mask = get_mask_for_df(df)
    mask_path = "{}.csv".format(str(df['image_id']))
    np.savetxt(mask_path, mask, delimiter=",")

In [ ]:
def get_mask_new(df):
    img = cv2.imread(os.path.join(image_path(df['image_id'])))
    mask = np.zeros((img.shape[0], img.shape[1], 2), dtype='float32')
    labels = df['labels']
    if isinstance(labels, str):
        labels = np.array(labels.split(' ')).reshape(-1, 5)
        for char, x, y, w, h in labels:
            x, y, w, h = int(x), int(y), int(w), int(h)
            if x + w >= img.shape[1] or y + h >= img.shape[0]:
                continue
            mask[y: y + h, x: x + w, 0] = 1
            radius = 6
            mask[y + h // 2 - radius: y + h // 2 + radius + 1, x + w // 2 - radius: x + w // 2 + radius + 1, 1] = 1
    return mask

def display_mask(mask, width=None, height=None):
    box = mask[:, :, 0]
    centers = mask[:, :, 1]
    
    if width:
        box = cv2.resize(box, (width, height))
        centers = cv2.resize(centers, (width, height))
    fig, axs = plt.subplots(1, 2, figsize=(8, 8))
    axs[0].imshow(box, interpolation='bilinear')
    axs[1].imshow(centers, interpolation='bilinear')
    plt.show()
    
df = train_df.iloc[0]
# show_image_with_labels(df)
mask= get_mask_new(df)
display_mask(mask)

In [ ]:
# def data_gen(dataframe, batch_size, img_width=256, img_height=256):
#     while (True):
#         img = np.zeros((batch_size, img_width, img_height, 3)).astype('float')
#         mask = np.zeros((batch_size, img_width, img_height, 1)).astype('float')
#         df_sample = dataframe.sample(batch_size)
#         for index, df in df_sample.iterrows():
#             i = 0
#             train_img = cv2.imread(os.path.join(image_path(df['image_id'])))
            
#             train_mask = get_mask_new(train_img, df['labels'])
#             train_mask = (train_mask[:,:,0]) / 255.
#             train_mask = cv2.resize(train_mask, (img_width, img_height))
#             train_mask = train_mask.reshape(img_width, img_height, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
#             mask[i] = train_mask
            
#             train_img = train_img / 255.
#             train_img =  cv2.resize(train_img, (img_width, img_height))# Read an image from folder and resize
#             img[i] = train_img #add to array - img[0], img[1], and so on.
#             i += 1

#         yield img, mask
        
# train, validate = train_test_split(train_df, test_size=0.2)
# train_generator = data_gen(train,16)
# validate_generator = data_gen(validate,16)

In [ ]:
class KuzushijiDataset(Sequence):
    def __init__(self, dataframe, batch_size, img_width=256, img_height=256, training=False):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.training = training
        self.index = 0

    def __len__(self):
        return math.ceil(len(df.index) / self.batch_size)

    def __getitem__(self, index):
        img = np.zeros((self.batch_size, self.img_width, self.img_height, 3)).astype('float')
        mask = np.zeros((self.batch_size, self.img_width, self.img_height, 1)).astype('float')
        if self.training:
            df_sample = self.dataframe.sample(self.batch_size)
        else:
            df_sample = self.dataframe[self.index:self.index+self.batch_size]
            
        i = 0
        for index, df in df_sample.iterrows():
            train_mask, train_img = self.get_img_and_mask(df)
            mask[i] = train_mask
            img[i] = train_img
            i = i + 1
            
        self.index = self.index + self.batch_size
        if (self.index + self.batch_size) > len(self.dataframe.index):
            self.index = 0

        return img, mask
    
    def get_img_and_mask(self, df):
        train_mask = get_mask_new(df) / 255.
        train_mask = cv2.resize(train_mask[:,:,0], (self.img_width, self.img_height))
        train_mask = train_mask.reshape(self.img_width, self.img_height, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
        train_img = cv2.imread(os.path.join(image_path(df['image_id']))) / 255.
        train_img =  cv2.resize(train_img, (self.img_width, self.img_height))# Read an image from folder and resize
        return train_mask, train_img
    
train, validate = train_test_split(train_df, test_size=0.1)
train_generator = KuzushijiDataset(train,16, training=True)
validate_generator = KuzushijiDataset(validate,16)

In [ ]:
class KuzushijiDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        # Add classes
        for i, name in enumerate(kuzushiji_to_detect):
            self.add_class("kuzushiji", i+1, name)
        
        # Add images 
        for i, row in df.iterrows():
            self.add_image("kuzushiji", 
                           image_id=row.name, 
                           path='../../input/kuzushiji-recognition/train_images/'+str(row.image_id)+".jpg", 
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [kuzushiji_to_detect[int(x)] for x in info['labels']]
        
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])
        

    def load_mask(self, image_id):
        info = self.image_info[image_id]
                
        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8)
            annotation = [int(x) for x in annotation.split(' ')]
            
            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1

            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
            
        return mask, np.array(labels)

In [ ]:
for img, mask in train_generator:
    for i in range(4):
        fig, axs = plt.subplots(1, 2, figsize=(8, 8))
        axs[0].imshow(img[i])
        axs[1].imshow(mask[i,:,:,0])
    break

In [ ]:
def loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
        denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))

        return tf.reshape(1 - numerator / denominator, (-1, 1, 1))

    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    input_layer = Input(shape=input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_layer)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = tf.keras.Model(inputs=input_layer, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy')
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

model = unet()

In [ ]:
%%time
model.reset_states()
steps = 1
epochs = 250
checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)
history = model.fit_generator(train_generator, 
                              steps_per_epoch=steps, 
                              epochs=epochs,
                              validation_data=validate_generator,
                              validation_steps=steps,
                              callbacks=[checkpointer])

In [ ]:

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [ ]:
def show_prediction(image_path, img_width=256, img_height=256):
    img = cv2.imread(os.path.join(image_path))
    height, width, channels = img.shape
    test_img = img / 255.
    test_img =  cv2.resize(test_img, (img_width, img_height))# Read an image from folder and resize
    test_img = np.expand_dims(test_img, axis=0)
    prediction_mask = model.predict(test_img) * 255.
#     display_mask(prediction_mask[0], width, height)
    prediction_mask = cv2.resize(prediction_mask[0], (width, height))
    print(prediction_mask)
    plt.imshow(prediction_mask, cmap='hot')
    plt.show()
    
    
    
df = validate.iloc[1]
show_prediction(image_path(df['image_id']))

In [ ]:
for img, mask in validate_generator:
    break
    

masks = model.predict(img) * 255.
for i in range(8):
    fig, axs = plt.subplots(1, 2, figsize=(8, 8))
    mask = cv2.resize(masks[i], (256, 256))
    axs[0].imshow(img[i])
    axs[1].imshow(mask)
plt.show()